In [1]:
import pandas as pd
import numpy as np
import os

import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy import inspect
from sqlalchemy.engine import reflection

In [2]:
df = pd.read_csv('data/Stations.csv')

In [3]:
# df.head()

In [4]:
engine = create_engine("sqlite:///mta.db.sqlite")

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)
# session connection to db
session = Session(engine)

In [5]:
insp = reflection.Inspector.from_engine(engine)
print(insp.get_table_names())

['sqlite_sequence', 'stationLocationData', 'turnStileData']


### Read in Fares csv and Turnstiles Fares Sations csv to a Pandas DataFrame
#### Join both data frames on Remote column
#### Remove Duplicate columns
#### Send to Sqlite DB


In [6]:
dfOne = pd.read_csv('data/turnstiles_fares_stations2.csv')
dfTwo = pd.read_csv('data/fares_190209.csv')

In [7]:
dfOne.head()

,UNIT,STATION,REMOTE,STATION,Station ID
0,R095,ASTORIA DITMARS,R095,DITMARS BLVD & 31ST STREET,1
1,R094,ASTORIA BLVD,R094,ASTORIA BLVD & 31ST STREET,2
2,R093,30 AV,R093,30TH AVENUE & 31ST STREET,3
3,R092,BROADWAY,R092,BROADWAY & 31ST STREET,4
4,R091,36 AV,R091,36TH AVENUE & 31ST STREET,5


In [8]:
dfTwo.head()

,REMOTE,STATION,FF,SEN/DIS,7-D AFAS UNL,30-D AFAS/RMF UNL,JOINT RR TKT,7-D UNL,30-D UNL,14-D RFM UNL,...,PATH 2-T,AIRTRAIN FF,AIRTRAIN 30-D,AIRTRAIN 10-T,AIRTRAIN MTHLY,STUDENTS,NICE 2-T,CUNY-120,CUNY-60,
0,R001,WHITEHALL STREET,59035,2875,415,1067,255,23503,37820,0,...,0,1069,0,0,0,2968,0,568,0,...
1,R003,CYPRESS HILLS,2682,147,16,41,0,2333,1325,0,...,0,30,0,0,0,544,0,45,0,...
2,R004,75TH STREET & ELDERTS LANE,6865,433,61,138,1,4839,3680,0,...,0,51,0,0,0,2420,0,53,0,...
3,R005,85TH STREET & FOREST PKWAY,7614,562,67,186,2,5007,4155,0,...,0,71,0,0,0,1249,0,100,0,...
4,R006,WOODHAVEN BOULEVARD,7835,477,57,169,0,6838,4732,0,...,0,64,0,0,0,1542,0,113,0,...


#### Merge Both tables on remote column


In [11]:
fareData = pd.merge(dfOne, dfTwo, on='REMOTE',how="outer")

##### Set index to Station ID

In [12]:
fareData.set_index('Station ID', inplace=True)

In [13]:
# fareData.head(1)

##### Remove duplicate columns

In [14]:
fareData.drop(columns=[' STATION_x', ' STATION_y'], inplace=True)

In [15]:
# fareData.head(1)

#### Take updated fare data and add to sqllite DB

In [16]:
fareData.to_sql('fareData', con=engine)

In [ ]:
# check to see that db has both tables
insp = reflection.Inspector.from_engine(engine)
print(insp.get_table_names())

### Read in turnstiles_190209.csv and turnstiles_fares_stations2.csv to a Pandas DataFrame
#### Join both data frames on Remote column
#### Remove Duplicate columns
#### Send to Sqlite DB


In [ ]:
dfOne = pd.read_csv('data/turnstiles_fares_stations2.csv')
dfThree = pd.read_csv('data/turnstiles_190209.csv')

In [ ]:
dfOne.head(1)

In [ ]:
dfThree.head(1)

In [ ]:
turnstileData = pd.merge(dfOne, dfThree, on='UNIT')

In [ ]:
turnstileData.head(1)

In [ ]:
turnstileData.drop(columns=['STATION_x', 'STATION_y'], inplace=True)

In [ ]:
turnstileData.set_index('Station ID',inplace=True)
turnstileData.head(1)

In [ ]:
# turnstileData.to_sql('turnStileData', con=engine)